In [1]:
%run ../ipynb_util_tars.py

In [2]:
%env WANDB_PROJECT=thesis

env: WANDB_PROJECT=thesis


In [3]:
%run ../ipynb_load_data_natural.py

{'SDG': Value(dtype='int64', id=None), 'ABSTRACT': Value(dtype='string', id=None), 'id': Value(dtype='string', id=None), 'sdg_desc_short': Value(dtype='string', id=None), 'sdg_desc_long': Value(dtype='string', id=None), '__index_level_0__': Value(dtype='int64', id=None)}
Example instance:	 {'SDG': 8, 'ABSTRACT': 'The scheme gives enterprises with business activity in Norway a tax credit on their R&D projects. The R&D content must be approved by the Research Council of Norway ex ante. In 2009, the cap on expenses per enterprise for intramural R&D projects increased to NOK 5.5 million (previously it was N0K 4 million), and NOK11 million (previously it was NOK 8 million) for projects conducted at an R&D institution.', 'id': None, 'sdg_desc_short': None, 'sdg_desc_long': None, '__index_level_0__': 492}
id2label: {0: '1', 1: '2', 2: '3', 3: '4', 4: '5', 5: '6', 6: '7', 7: '8', 8: '9', 9: '10', 10: '11', 11: '12', 12: '13', 13: '14', 14: '15', 15: '16', 16: '17'}
label2id: {'1': 0, '2': 1, '

# LLaMA3 8B

In [4]:
from transformers import LlamaTokenizer, AutoTokenizer

# base model
HF_MODEL_NAME = "meta-llama/Meta-Llama-3-8B"
# final model
MODEL_NAME = f"{HF_MODEL_NAME}-ft-{DATASET_TYPE.value}" + ("-lower" if SHOULD_LOWER else "")

tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
# print(tokenizer.pad_token, tokenizer.eos_token)


encoded_dataset = dataset.map(preprocess_data(tokenizer), batched=True, remove_columns=dataset["train"].column_names)
encoded_dataset.set_format("torch")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/dill/_dill.py:414: PicklingWarning: Cannot locate reference to <enum 'DatasetType'>.
  StockPickler.save(self, obj, save_persistent_id)
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/dill/_dill.py:414: PicklingWarning: Cannot pickle <enum 'DatasetType'>: __main__.DatasetType has recursive self-references that trigger a RecursionError.
  StockPickler.save(self, obj, save_persistent_id)


Map:   0%|          | 0/630 [00:00<?, ? examples/s]

Map:   0%|          | 0/271 [00:00<?, ? examples/s]

## Model

In [5]:
from transformers.data import DataCollatorWithPadding
from transformers.models.llama import LlamaForSequenceClassification
from peft import LoraConfig, TaskType, get_peft_model
from transformers import BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    #bnb_4bit_use_double_quant=True,
    #bnb_4bit_quant_type="nf4",
    #bnb_4bit_compute_dtype=torch.bfloat16
)

model = LlamaForSequenceClassification.from_pretrained(
    HF_MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_config,
    #num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    token=HF_TOKEN
).bfloat16()
model.config.pad_token_id = tokenizer.pad_token_id
# model.resize_token_embeddings(len(tokenizer))

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=32,
    lora_alpha=32,
    lora_dropout=0.1
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

data_collator = DataCollatorWithPadding(tokenizer)

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import TrainingArguments, Trainer, EvalPrediction

BATCH_SIZE = 4
METRIC_NAME = "accuracy"

args = TrainingArguments(
    f"{CHECKPOINT_PATH}/final/{MODEL_NAME}",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=4e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=20,
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model=METRIC_NAME,
    seed=SEED,
    report_to="wandb",
    run_name=MODEL_NAME
)

def compute_metrics(pred: EvalPrediction):
    labels = pred.label_ids
    accuracy = accuracy_score(labels, pred.predictions.argmax(-1))
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred.predictions.argmax(-1), average="weighted")
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dvdblk (ngmi). Use `wandb login --relogin` to force relogin


/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.705300,2.471344,0.269373,0.277453,0.269373,0.248446
2,1.682500,1.310064,0.616236,0.705826,0.616236,0.619484
3,0.698100,0.935476,0.760148,0.743422,0.760148,0.746499
4,0.311500,0.884076,0.771218,0.781707,0.771218,0.767822
5,0.095800,1.047368,0.778598,0.784195,0.778598,0.776484
6,0.010700,1.015885,0.785978,0.788156,0.785978,0.783274
7,0.000600,1.022824,0.804428,0.807352,0.804428,0.803850
8,0.000100,0.995230,0.811808,0.810270,0.811808,0.807858
9,0.000100,0.998517,0.811808,0.807246,0.811808,0.806408
10,0.000100,0.998036,0.811808,0.803256,0.811808,0.804517


/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/user/dbielik/msc-thesis/venv/lib/python3.1

TrainOutput(global_step=3160, training_loss=0.3252507964172695, metrics={'train_runtime': 5084.7855, 'train_samples_per_second': 2.478, 'train_steps_per_second': 0.621, 'total_flos': 2.706933396013056e+17, 'train_loss': 0.3252507964172695, 'epoch': 20.0})

In [ ]:
trainer.evaluate()

/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 1.0286328792572021,
 'eval_accuracy': 0.8228782287822878,
 'eval_precision': 0.8172039432558323,
 'eval_recall': 0.8228782287822878,
 'eval_f1': 0.8167687578398126,
 'eval_runtime': 30.7581,
 'eval_samples_per_second': 8.811,
 'eval_steps_per_second': 2.211,
 'epoch': 20.0}

In [ ]:
from sklearn.metrics import classification_report

trainer.model.eval()

# manual evaluation to show classifcation_report
true_labels = []
logits = []

for batch in encoded_dataset["test"]:
    batch = {k: v.to(trainer.args.device).unsqueeze(0) for k, v in batch.items()}
    label = batch.pop("label")

    # Forward pass
    with torch.no_grad():
        out = trainer.model(**batch)

    true_labels.append(label.item())
    logits.extend(out.logits.tolist())

probabilites = torch.nn.functional.softmax(torch.tensor(logits), dim=-1)
pred_labels = torch.argmax(probabilites, dim=-1).tolist()

report = classification_report(true_labels, pred_labels, target_names=[f"SDG {id2label[i]}" for i in range(len(labels))], digits=4)

/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [ ]:
print(report)

              precision    recall  f1-score   support

       SDG 1     0.7778    0.8235    0.8000        17
       SDG 2     0.8000    0.9412    0.8649        17
       SDG 3     0.8235    0.8235    0.8235        17
       SDG 4     0.8889    0.9412    0.9143        17
       SDG 5     0.7368    0.8235    0.7778        17
       SDG 6     0.9412    1.0000    0.9697        16
       SDG 7     0.9231    0.7500    0.8276        16
       SDG 8     0.7000    0.8235    0.7568        17
       SDG 9     0.7222    0.7647    0.7429        17
      SDG 10     0.5000    0.2353    0.3200        17
      SDG 11     0.8889    0.9412    0.9143        17
      SDG 12     0.8571    0.7059    0.7742        17
      SDG 13     0.8235    0.8235    0.8235        17
      SDG 14     1.0000    1.0000    1.0000        17
      SDG 15     0.8824    0.8824    0.8824        17
      SDG 16     0.6842    0.7647    0.7222        17
      SDG 17     0.0000    0.0000    0.0000         1

    accuracy              

In [ ]:
sample_sentence = """Ensure access to affordable, reliable, sustainable and modern energy for all """
probs = torch.nn.functional.softmax(model(**tokenizer(sample_sentence, return_tensors="pt", truncation=True, max_length=512).to(model.device)).logits, dim=-1)

for i, prob in enumerate(probs[0]):
    print(f"Label {model.config.id2label[i]}: {prob.item()}")

print(model.config.id2label)
print(model.config.label2id)

/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Label LABEL_0: 0.0002956390380859375
Label LABEL_1: 0.000492095947265625
Label LABEL_2: 0.001251220703125
Label LABEL_3: 0.0002765655517578125
Label LABEL_4: 0.00010347366333007812
Label LABEL_5: 0.00025177001953125
Label LABEL_6: 0.21875
Label LABEL_7: 0.08544921875
Label LABEL_8: 0.435546875
Label LABEL_9: 0.08447265625
Label LABEL_10: 0.05126953125
Label LABEL_11: 0.0869140625
Label LABEL_12: 0.019287109375
Label LABEL_13: 0.0001678466796875
Label LABEL_14: 0.015625
Label LABEL_15: 7.43865966796875e-05
Label LABEL_16: 0.000324249267578125
{0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2', 3: 'LABEL_3', 4: 'LABEL_4', 5: 'LABEL_5', 6: 'LABEL_6', 7: 'LABEL_7', 8: 'LABEL_8', 9: 'LABEL_9', 10: 'LABEL_10', 11: 'LABEL_11', 12: 'LABEL_12', 13: 'LABEL_13', 14: 'LABEL_14', 15: 'LABEL_15', 16: 'LABEL_16'}
{'LABEL_0': 0, 'LABEL_1': 1, 'LABEL_2': 2, 'LABEL_3': 3, 'LABEL_4': 4, 'LABEL_5': 5, 'LABEL_6': 6, 'LABEL_7': 7, 'LABEL_8': 8, 'LABEL_9': 9, 'LABEL_10': 10, 'LABEL_11': 11, 'LABEL_12': 12, 'LABEL_13'